# C

In [101]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [102]:
# compile to a shared library by piping C_code to gcc:
using Libdl
const Clib = tempname()

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [103]:
c_sum(rand(10))

5.415699659076963

In [105]:
x = rand(10)
@btime c_sum($x);

  3.733 ns (0 allocations: 0 bytes)


# Python

In [6]:
using PyCall

In [7]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/crstnbr/opt/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>

In [60]:
np.linalg.eigvals(np.random.rand(5,5))

5-element Array{Float64,1}:
  3.3640844258821025
 -0.41966309080077363
 -0.04857463403251538
  0.2167722366676619
  0.670248036886041

In [64]:
M = rand(5,5)
np.linalg.eigvals(M)

5-element Array{Complex{Float64},1}:
    2.555080179412813 + 0.0im
  -0.9171259377677111 + 0.0im
 -0.12774373739007613 + 0.0im
   0.5511822315013549 + 0.0859843279752961im
   0.5511822315013549 - 0.0859843279752961im

In [94]:
py"""
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [98]:
py_sinpi(x) = py"sinpi"(x)

py_sinpi (generic function with 1 method)

In [99]:
py_sinpi(10)

-1.2246467991473533e-15

In [132]:
@btime py_sinpi(10);

  10.086 μs (10 allocations: 256 bytes)


# Mixing everything

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [127]:
x = rand(10);

In [128]:
py_sinpi(real(c_sum(x)))

0.7636821843838295

In [129]:
@btime py_sinpi(real(c_sum($x)));

  9.350 μs (8 allocations: 224 bytes)


See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).